# Decision Tree Classif

In [ ]:
# import dataset for testing
import numpy as np
from numpy import shape
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
data = datasets.load_breast_cancer()
X, y = data['data'], data['target']
X, y

(array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
shape(X_train), shape(y_train)

((455, 30), (455,))

In [4]:
# Define Node object
class Node:
    def __init__(self, feature = None, threshold = None, left = None, right = None, * , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value =value

# Define whether Node is leaf
    def is_leaf(self):
        return self.value is not None

In [8]:
# Define DecisionTree Classifier
class DecisionTree:
    def __init__(self, max_depth = 100, min_samples_split = 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    # Stoping criteria
    def _is_finished(self, depth):
        if (depth >= self.max_depth or self.n_class_labels == 1 or self.n_samples < self.min_samples_split):
            return True
        
        return False
    
    # Tree build-up
    def _build_tree(self, X, y, depth = 0):
        # Get Data 
        self.n_samples, self.n_features = X.shape
        self.n_class_labels = len(np.unique(y))
        
        # Stoping criteria
        if self._is_finished(depth):
            most_common_label = np.argmax(np.bincount(y))
            return Node(value = most_common_label)
        
        rnd_feats = np.random.choice(self.n_features, self.n_features, replace = False)
        best_feat, best_thresh = self._best_split(X, y, rnd_feats)
        left_idx, right_idx = self._create_split(X[:, best_feat], best_thresh)
        
        left_child = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_child = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        
        return Node(best_feat, best_thresh, left_child, right_child)
    
    # Invoke recursion function _build_tree, fit Data
    def fit(self, X, y):
        self.root = self._build_tree(X, y)
    
    # Define entropy
    def _entropy(self, y):
        proportion = np.bincount(y) / len(y)
        
        entropy = 0
        for prop in proportion:
            if prop > 0:
                entropy += -prop * np.log2(prop)
        
        return entropy
    
    # Define decision rule
    def _create_split(self, X, threshold):
        left_idx = np.argwhere(X <= threshold).flatten()
        right_idx= np.argwhere(X > threshold).flatten()
        
        return left_idx, right_idx
    
    # Define information gain, using loss function entropy
    def _information_gain(self, X, y, threshold):
        parent_loss = self._entropy(y)
        left_idx, right_idx = self._create_split(X, threshold)
        n, n_left, n_right = len(y), len(left_idx), len(right_idx)
        
        if n_left == 0 or n_right == 0:
            return 0
        
        child_loss = (n_left / n) * self._entropy(y[left_idx]) + (n_right / n) * self._entropy(y[right_idx])
        
        return parent_loss - child_loss
    
    # Define Decision making-rule
    # Maximum information Gain == Minimize loss function
    def _best_split(self, X, y, features):
        split = {'score': -1, 'feat': None, 'threshold': None}
        
        for feat in features:
            X_feat = X[:, feat]
            thresholds = np.unique(X_feat)
            
            for thresh in thresholds:
                score = self._information_gain(X_feat, y, thresh)
                
                if score > split['score']:
                    split['score'] = score
                    split['feat'] = feat
                    split['threshold'] = thresh
        
        return split['feat'], split['threshold']
    
    # Define prediction
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)
    
    # Invoke traversal
    def predict(self, X):
        predictions = [self._traverse_tree(x, self.root) for x in X]
        
        return np.array(predictions)

In [9]:
# Define accuracy score
def accuracy_score(y_hat, y):
    accuracy = np.sum(y_hat == y) / len(y)
    
    return accuracy

In [10]:
model = DecisionTree(max_depth = 10)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_pred, y_test)

print('Accuracy:', acc)

Accuracy: 0.9385964912280702


In [11]:
y_pred

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0], dtype=int64)

<b> END </b>